# Installation librairies

In [ ]:
#

# Import librairies

In [ ]:
import pandas as pd
import requests
import os

# DATA IMPORT:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# chemins
path = '/content/drive/MyDrive/ICU/'
path_down = '/content/drive/MyDrive/ICU/Download_data/'
path_analyse = '/content/drive/MyDrive/ICU/Analyse/'
path_carto = '/content/drive/MyDrive/ICU/Carto/'

# Nettoyage dataset

In [ ]:
# Lister les datasets
List_dataset = [f for f in os.listdir(path_down) if os.path.isfile(os.path.join(path_down, f))]

# Fichiers à supprimer de la liste
files_to_remove = ['Liste_station.csv']

# Utilisation d'une boucle pour supprimer les fichiers de la liste s'ils existent
for file in files_to_remove:
    if file in List_dataset:
        List_dataset.remove(file)

# Supprimer le '.csv' dans les noms de fichiers
List_dataset = [f.replace('.csv', '') for f in List_dataset]

print(List_dataset)

['00-station-meteo-toulouse-valade', '01-station-meteo-toulouse-meteopole', '02-station-meteo-toulouse-marengo', '03-station-meteo-toulouse-busca', '04-station-meteo-toulouse-ile-empalot', '05-station-meteo-toulouse-life-hall-1', '05-station-meteo-toulouse-nakache', '07-station-meteo-toulouse-avenue-de-grande-bretagne', '08-station-meteo-toulouse-basso-cambo', '09-station-meteo-toulouse-la-salade', '10-station-meteo-castelginest-ecole', '11-station-meteo-toulouse-soupetard', '12-station-meteo-toulouse-montaudran', '13-station-meteo-toulouse-pech-david', '14-station-meteo-toulouse-centre-pierre-potier', '15-station-meteo-l-union-ecole', '17-station-meteo-fenouillet-foyer', '18-station-meteo-brax-ecole', '19-station-meteo-mondouzil-mairie', '20-station-meteo-mondonville-ecole', '21-station-meteo-cugnaux-general-de-gaulle', '22-station-meteo-colomiers-za-perget', '23-station-meteo-pibrac-bouconne-centre-equestre', '24-station-meteo-colomiers-zi-enjacca', '25-station-meteo-tournefeuille-re

## Import Dataset

In [ ]:
# Initialiser un dictionnaire pour stocker les DataFrames
dfs = {}

# Liste des colonnes à conserver et organiser
colonnes_a_conserver = [
    'heure_utc', 'id', 'type_de_station', 'temperature', 'humidite', 'pression',
    'pluie', 'pluie_intensite_max', 'direction_du_vecteur_vent_moyen',
    'force_moyenne_du_vecteur_vent', 'direction_du_vecteur_de_vent_max',
    'direction_du_vecteur_de_rafale_de_vent_max', 'force_rafale_max'
]

# Dictionnaire pour spécifier les types de données des colonnes
dtype_dict = {
    'heure_utc': 'str',
    'station_name': 'str',
    'id': 'str',
    'type_de_station': 'str',
    'temperature': 'float64',
    'humidite': 'float64',
    'pression': 'float64',
    'pluie': 'float64',
    'pluie_intensite_max': 'float64',
    'direction_du_vecteur_vent_moyen': 'float64',
    'force_moyenne_du_vecteur_vent': 'float64',
    'direction_du_vecteur_de_vent_max': 'float64',
    'direction_du_vecteur_de_rafale_de_vent_max': 'float64',
    'force_rafale_max': 'float64',
}

# Boucle pour créer un DataFrame pour chaque fichier dans List_dataset
for dataset in List_dataset:
    file_path = os.path.join(path_down, dataset+'.csv')
    try:
        # Lire chaque fichier CSV et stocker dans le dictionnaire avec les types de données spécifiés
        dfs[f"df_{dataset}"] = pd.read_csv(file_path, sep=';', dtype=dtype_dict, low_memory=False)

        # Vérifier et renommer les colonnes si nécessaire
        if 'dd_mm_yy_hh_mm' in dfs[f"df_{dataset}"].columns:
            dfs[f"df_{dataset}"].rename(columns={'dd_mm_yy_hh_mm': 'heure_utc'}, inplace=True)
        if 'temperature_en_degre_c' in dfs[f"df_{dataset}"].columns:
            dfs[f"df_{dataset}"].rename(columns={'temperature_en_degre_c': 'temperature'}, inplace=True)

        # Vérifier les colonnes présentes et organiser selon celles disponibles
        colonnes_disponibles = [col for col in colonnes_a_conserver if col in dfs[f"df_{dataset}"].columns]
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"][colonnes_disponibles]

        # Supprimer les doublons dans les colonnes
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"].loc[:, ~dfs[f"df_{dataset}"].columns.duplicated()]

        # Supprimer les lignes avec des valeurs manquantes
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"].dropna()

        # Supprimer les doublons
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"].drop_duplicates()

        # Identifier les valeurs aberrantes pour la colonne 'temperature'
        q1 = dfs[f"df_{dataset}"]['temperature'].quantile(0.25)
        q3 = dfs[f"df_{dataset}"]['temperature'].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Filtrer les valeurs aberrantes
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"][(dfs[f"df_{dataset}"]['temperature'] >= lower_bound) & (dfs[f"df_{dataset}"]['temperature'] <= upper_bound)]

        # Convertir la colonne 'heure_utc' en type datetime
        if 'heure_utc' in dfs[f"df_{dataset}"].columns:
            dfs[f"df_{dataset}"].loc[:, 'heure_utc'] = pd.to_datetime(dfs[f"df_{dataset}"].loc[:, 'heure_utc'], utc=True)

        # Trier par 'heure_utc'
        if 'heure_utc' in dfs[f"df_{dataset}"].columns:
            dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"].sort_values(by='heure_utc')

        # Droper les lignes où 'id' est différent de la valeur la plus fréquente
        if 'id' in dfs[f"df_{dataset}"].columns:
            id_plus_frequent = dfs[f"df_{dataset}"]['id'].mode()[0]  # Trouver l'ID le plus fréquent
            dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"][dfs[f"df_{dataset}"]['id'] == id_plus_frequent]  # Filtrer selon cet ID

        # Ajouter le nom de la station
        dfs[f"df_{dataset}"]['station_name'] = f"{dataset}"
        # Placer la colonne station_name en premier
        cols = ['station_name'] + [col for col in dfs[f"df_{dataset}"].columns if col != 'station_name']
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"][cols]

        # Réinitialiser les index après toutes les opérations
        dfs[f"df_{dataset}"] = dfs[f"df_{dataset}"].reset_index(drop=True)

        print(f"DataFrame créé, organisé et trié pour {dataset}")

    except Exception as e:
        print(f"Erreur sur {dataset} : {e}")

DataFrame créé, organisé et trié pour 00-station-meteo-toulouse-valade
DataFrame créé, organisé et trié pour 01-station-meteo-toulouse-meteopole
DataFrame créé, organisé et trié pour 02-station-meteo-toulouse-marengo
DataFrame créé, organisé et trié pour 03-station-meteo-toulouse-busca
DataFrame créé, organisé et trié pour 04-station-meteo-toulouse-ile-empalot
DataFrame créé, organisé et trié pour 05-station-meteo-toulouse-life-hall-1
DataFrame créé, organisé et trié pour 05-station-meteo-toulouse-nakache
DataFrame créé, organisé et trié pour 07-station-meteo-toulouse-avenue-de-grande-bretagne
DataFrame créé, organisé et trié pour 08-station-meteo-toulouse-basso-cambo
DataFrame créé, organisé et trié pour 09-station-meteo-toulouse-la-salade
DataFrame créé, organisé et trié pour 10-station-meteo-castelginest-ecole
DataFrame créé, organisé et trié pour 11-station-meteo-toulouse-soupetard
DataFrame créé, organisé et trié pour 12-station-meteo-toulouse-montaudran
DataFrame créé, organisé e

In [ ]:
# Concaténer tous les DataFrames dans df_Stations_All
df_Stations_All = pd.concat(dfs.values(), ignore_index=True)

In [ ]:
# Exporter df_Stations_All en un fichier CSV
output_path = os.path.join(path_analyse, 'Stations_All.csv')  # Spécifier le chemin de sauvegarde du fichier CSV
df_Stations_All.to_csv(output_path, sep=';', index=False)

print(f"DataFrame exporté en CSV à : {output_path}")

DataFrame exporté en CSV à : /content/drive/MyDrive/ICU/Analyse/Stations_All.csv


In [ ]:
display(df_Stations_All.head(5))
df_Stations_All.describe()

,station_name,heure_utc,id,type_de_station,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max
0,00-station-meteo-toulouse-valade,2020-06-08 13:15:00+00:00,0,ISS,16.1,74.0,99500.0,0.0,0.0,0.0,0.0,6.0,135.0,3.0
1,00-station-meteo-toulouse-valade,2020-06-08 13:30:00+00:00,0,ISS,17.0,71.0,99500.0,0.0,0.0,58.0,2.0,3.0,67.5,8.0
2,00-station-meteo-toulouse-valade,2020-06-08 13:45:00+00:00,0,ISS,17.0,72.0,99500.0,0.0,0.0,74.0,2.0,4.0,90.0,11.0
3,00-station-meteo-toulouse-valade,2020-06-08 14:00:00+00:00,0,ISS,17.4,69.0,99500.0,0.0,0.0,0.0,3.0,4.0,90.0,11.0
4,00-station-meteo-toulouse-valade,2020-06-08 14:15:00+00:00,0,ISS,18.3,63.0,99500.0,0.0,0.0,0.0,2.0,11.0,247.5,13.0


,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max
count,4.900627e+06,4.900627e+06,4.900627e+06,4.900627e+06,4.510943e+06,4.900627e+06,4.805645e+06,4.900627e+06,4.900627e+06,4.900627e+06
mean,1.373062e+01,6.866036e+01,9.169135e+04,1.018968e-02,7.699587e-03,8.395470e+01,2.824855e+00,6.214220e+00,1.398200e+02,9.026099e+00
std,1.249969e+01,2.071647e+01,2.702404e+04,2.375630e-01,6.438484e-02,1.112579e+02,3.684392e+00,5.308435e+00,1.194398e+02,9.105269e+00
min,-5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.300000e+00,5.700000e+01,9.910000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.440000e+01,7.200000e+01,9.980000e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,6.000000e+00,1.350000e+02,6.000000e+00
75%,2.050000e+01,8.500000e+01,1.003000e+05,0.000000e+00,0.000000e+00,1.520000e+02,4.000000e+00,1.100000e+01,2.475000e+02,1.400000e+01
max,4.040000e+01,1.000000e+02,1.155000e+05,8.500000e+01,3.000000e+00,3.600000e+02,5.100000e+01,1.500000e+01,3.375000e+02,1.640000e+02
